In [1]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
csv_files = [
    {'file_path': '/Users/giorgioramirez/Desktop/CreditCardRecordsDB/AMEX_CSV/Plat23.csv', 'Card_Used': 'PLATINUM'},
    {'file_path': '/Users/giorgioramirez/Desktop/CreditCardRecordsDB/AMEX_CSV/Plat24.csv', 'Card_Used': 'PLATINUM'},
    {'file_path': '/Users/giorgioramirez/Desktop/CreditCardRecordsDB/AMEX_CSV/Hilton.csv', 'Card_Used': 'HILTON ASPIRE'},
    {'file_path': '/Users/giorgioramirez/Desktop/CreditCardRecordsDB/AMEX_CSV/Gold24.csv', 'Card_Used': 'GOLD'},
    {'file_path': '/Users/giorgioramirez/Desktop/CreditCardRecordsDB/AMEX_CSV/BizPlat24.csv', 'Card_Used': 'BUSINESS PLATINUM'},
    {'file_path': '/Users/giorgioramirez/Desktop/CreditCardRecordsDB/AMEX_CSV/BizGold24.csv', 'Card_Used': 'BUSINESS GOLD'},
    {'file_path': '/Users/giorgioramirez/Desktop/CreditCardRecordsDB/AMEX_CSV/BizGold23.csv', 'Card_Used': 'BUSINESS GOLD'},
    {'file_path': '/Users/giorgioramirez/Desktop/CreditCardRecordsDB/AMEX_CSV/BBP22.csv', 'Card_Used': 'BLUE BUSINESS PLUS'},
    {'file_path': '/Users/giorgioramirez/Desktop/CreditCardRecordsDB/AMEX_CSV/BBP23.csv', 'Card_Used': 'BLUE BUSINESS PLUS'},
    {'file_path': '/Users/giorgioramirez/Desktop/CreditCardRecordsDB/AMEX_CSV/BBP24.csv', 'Card_Used': 'BLUE BUSINESS PLUS'},
   ]

 Read csv files preprocess add to list

In [7]:
df_list = []

for csv_info in csv_files:
    df = pd.read_csv(csv_info['file_path'])
    
    df['Card_Used'] = csv_info['Card_Used']
    
    if 'City/State' in df.columns:
        df[['City', 'State']] = df['City/State'].str.split('\n', expand=True)
        df.drop(columns=['City/State'], inplace=True)
    
    df = df.rename(columns={'Zip Code': 'Zip_Code'})
    
    df = df[['Date', 'Description', 'Amount', 'Address', 'City', 'State', 'Zip_Code', 'Country', 'Category', 'Card_Used']]
    
    df_list.append(df)

Convert list of dataframes to df

In [14]:
df = pd.concat(df_list, ignore_index=True)

In [16]:
df.head()

,Date,Description,Amount,Address,City,State,Zip_Code,Country,Category,Card_Used
0,12/27/2023,MOBILE PAYMENT - THANK YOU,-40.48,NaN,NaN,NaN,NaN,NaN,NaN,PLATINUM
1,12/27/2023,SAKS SAN ANTONIO 657SAN ANTONIO TX,50.00,7400 SAN PEDRO AVE STE 650,SAN ANTONIO,78216,TX,UNITED STATES,Merchandise & Supplies-Department Stores,PLATINUM
2,12/24/2023,DISNEY PLUS BURBANK CA,26.98,500 SOUTH BUENA VIS ST,BURBANK,91521,CA,UNITED STATES,Business Services-Office Supplies,PLATINUM
3,12/24/2023,DISNEY PLUS ADY BURBANK,-20.00,500 SOUTH BUENA VIS ST,BURBANK,91521,CA,UNITED STATES,Business Services-Office Supplies,PLATINUM
4,12/13/2023,WALMART PLUS MONTHLYBENTONVILLE,-12.95,702 SOUTH WEST 8TH ST,BENTONVILLE,72712,AR,UNITED STATES,Merchandise & Supplies-Internet Purchase,PLATINUM


In [18]:
df['Category'].unique()

array([nan, 'Merchandise & Supplies-Department Stores',
       'Business Services-Office Supplies',
       'Merchandise & Supplies-Internet Purchase', 'Travel-Airline',
       'Entertainment-General Attractions',
       'Transportation-Taxis & Coach', 'Travel-Lodging',
       'Restaurant-Bar & Café', 'Other-Government Services',
       'Travel-Travel Agencies',
       'Merchandise & Supplies-Electronics Stores', 'Other-Miscellaneous',
       'Merchandise & Supplies-General Retail',
       'Merchandise & Supplies-Groceries', 'Restaurant-Restaurant',
       'Business Services-Health Care Services',
       'Fees & Adjustments-Fees & Adjustments', 'Other-Education',
       'Business Services-Mailing & Shipping', 'Transportation-Fuel',
       'Merchandise & Supplies-Wholesale Stores',
       'Transportation-Auto Services', 'Business Services-Other Services',
       'Merchandise & Supplies-Clothing Stores',
       'Entertainment-Associations',
       'Communications-Cable & Internet Comm',
 

Reversed mapping dictionary with simplified category as a key and original categories as a list of values to consolidate categories. Customized certain categories based on known credit card specific earnings. List comprehension to map the list values to their keys.

In [21]:
simplified_category_mapping = {
    'Retail': [
        'Merchandise & Supplies-Department Stores',
        'Merchandise & Supplies-Internet Purchase',
        'Merchandise & Supplies-Electronics Stores',
        'Merchandise & Supplies-General Retail',
        'Merchandise & Supplies-Wholesale Stores',
        'Merchandise & Supplies-Clothing Stores',
        'Merchandise & Supplies-Computer Supplies',
        'Merchandise & Supplies-Pharmacies',
        'Merchandise & Supplies-Mail Order',
        'Merchandise & Supplies-Sporting Goods Stores'
    ],
    'Business_Services': [
        'Business Services-Office Supplies',
        'Business Services-Health Care Services',
        'Business Services-Mailing & Shipping',
        'Business Services-Other Services',
        'Business Services-Insurance Services',
        'Business Services-Professional Services',
        'Business Services-Internet Services'
    ],
    'Travel_Airline': [
        'Travel-Airline'
    ],
    'Travel': [
        'Travel-Lodging',
        'Travel-Travel Agencies'
    ],
    'Dining': [
        'Restaurant-Bar & Café',
        'Restaurant-Restaurant'
    ],
    'Transportation': [
        'Transportation-Taxis & Coach',
        'Transportation-Auto Services',
        'Transportation-Other Transportation',
        'Transportation-Rail Services',
        'Transportation-Parking Charges'
    ],
    'Fuel': [
        'Transportation-Fuel'
    ],
    'Entertainment': [
        'Entertainment-General Attractions',
        'Entertainment-Associations',
        'Entertainment-Other Entertainment',
        'Entertainment-General Events'
    ],
    'Communications': [
        'Communications-Cable & Internet Comm',
        'Communications-Telephone Comm'
    ],
    'Miscellaneous': [
        'Other-Government Services',
        'Other-Miscellaneous',
        'Other-Education',
        'Other-Charities',
        'Fees & Adjustments-Fees & Adjustments'
    ],
    'Groceries': [
        'Merchandise & Supplies-Groceries'
    ]
}

reverse_mapping = {item: simplified for simplified, items in simplified_category_mapping.items() for item in items}

Create new Categories column and drop old one.

In [27]:
df['Standardized_Category'] = df['Category'].map(reverse_mapping).fillna('Unknown')

In [29]:
df.head()

,Date,Description,Amount,Address,City,State,Zip_Code,Country,Category,Card_Used,Standardized_Category
0,12/27/2023,MOBILE PAYMENT - THANK YOU,-40.48,NaN,NaN,NaN,NaN,NaN,NaN,PLATINUM,Unknown
1,12/27/2023,SAKS SAN ANTONIO 657SAN ANTONIO TX,50.00,7400 SAN PEDRO AVE STE 650,SAN ANTONIO,78216,TX,UNITED STATES,Merchandise & Supplies-Department Stores,PLATINUM,Retail
2,12/24/2023,DISNEY PLUS BURBANK CA,26.98,500 SOUTH BUENA VIS ST,BURBANK,91521,CA,UNITED STATES,Business Services-Office Supplies,PLATINUM,Business_Services
3,12/24/2023,DISNEY PLUS ADY BURBANK,-20.00,500 SOUTH BUENA VIS ST,BURBANK,91521,CA,UNITED STATES,Business Services-Office Supplies,PLATINUM,Business_Services
4,12/13/2023,WALMART PLUS MONTHLYBENTONVILLE,-12.95,702 SOUTH WEST 8TH ST,BENTONVILLE,72712,AR,UNITED STATES,Merchandise & Supplies-Internet Purchase,PLATINUM,Retail


In [31]:
df = df.drop(columns=['Category'])
df = df.rename(columns={'Standardized_Category': 'Category'})

In [43]:
df['Category'].unique()

array(['BLUE BUSINESS PLUS', 'HILTON ASPIRE', 'PLATINUM', 'BUSINESS GOLD',
       'GOLD', 'BUSINESS PLATINUM'], dtype=object)

In [45]:
df['Card_Used'].unique()

array(['BLUE BUSINESS PLUS', 'HILTON ASPIRE', 'PLATINUM', 'BUSINESS GOLD',
       'GOLD', 'BUSINESS PLATINUM'], dtype=object)

Convert Date column to datatime type

In [36]:
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

In [38]:
df = df.sort_values(by='Date', ascending=True)

In [40]:
df.head()

,Date,Description,Amount,Address,City,State,Zip_Code,Country,Card_Used,Category
759,2022-09-02,AUTOPAY PAYMENT - THANK YOU,-2.88,NaN,NaN,NaN,NaN,NaN,BLUE BUSINESS PLUS,Unknown
489,2022-09-12,AplPay PANDA EXPRESSSAN ANTONIO TX,11.10,834 NORTH WEST LOOP 410,SAN ANTONIO,TX,78216-5602,UNITED STATES,HILTON ASPIRE,Dining
488,2022-09-13,Southwest Airlines DALLAS TX,66.14,PO BOX 36611,DALLAS,TX,75235,UNITED STATES,HILTON ASPIRE,Travel_Airline
487,2022-09-13,MOBILE PAYMENT - THANK YOU,-92.05,NaN,NaN,NaN,NaN,NaN,HILTON ASPIRE,Unknown
486,2022-09-16,AMEX Airline Fee Reimbursement,-66.14,PO BOX 36611,DALLAS,TX,75235,UNITED STATES,HILTON ASPIRE,Travel_Airline


Initialize engine connect to MySQLWorkbench and load transactions

In [17]:
user = 'root'
password = '*******'
host = 'localhost:3306'
database = 'CreditCardRecords'
table = 'Transactions'

In [18]:
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}')

In [19]:
table_name = 'Transactions'  

df.to_sql(table_name, con=engine, if_exists='append', index=False)

895